## BERTScore Implementation

In [1]:
import torch
print(torch.__version__)
# !pip install torch

1.4.0


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
!pip install bert_score -q
import bert_score
bert_score.__version__

'0.3.2'

In [4]:
import pandas as pd
import numpy as np
import os

filename = 'gdrive/My Drive/Colab Notebooks/Data/asap_sas.tsv'
df = pd.read_csv(filename, delimiter='\t')

can1 = df.loc[(df['EssaySet'] == 3)]
can1.reset_index(drop=True, inplace=True)
cands = list(can1['EssayText'])
print("Candidate Sentences: ", len(cands))
# print(cands[0])

ref1 = df.loc[(df['EssaySet'] == 3) & (df['Score1'] == 2)]
ref1.reset_index(drop=True, inplace=True)
refs = list(ref1['EssayText'])
print("Reference Sentences: ", len(refs))
# print(refs[0])

Candidate Sentences:  1891
Reference Sentences:  441


In [0]:
# broadcasting refs so that len(cands) == len(refs)
refs = [refs] * len(cands)
assert len(cands) == len(refs)

In [6]:
print(len(cands), type(cands))
print(len(refs), type(refs))

1891 <class 'list'>
1891 <class 'list'>


In [0]:
from bert_score import score

sol = score(cands, refs, model_type=None, num_layers=None, verbose=True,
            idf=True, device=None, batch_size=64, nthreads=4, all_layers=False,
            lang="en", return_hash=False, rescale_with_baseline=True)

preparing IDF dict...
done in 232.01 seconds
calculating scores...
computing bert embedding.



computing greedy matching.


In [0]:
P = sol[0]
R = sol[1]
F1 = sol[2]
print(len(F1))

In [0]:
print(can1.columns)
odf = pd.DataFrame()
odf['Id'] = can1['Id']
odf['Similarity'] = pd.DataFrame(F1)
odf['Score1'] = can1['Score1']
odf.reset_index(drop=True, inplace=True)
print(odf.columns)

odf.to_csv("outs.tsv", sep="\t", index=False, header=True)

In [0]:
# import matplotlib.pyplot as plt
# plt.hist(F1, bins=20)
# plt.show()

In [0]:
# from bert_score import plot_example
# cand = cands[0]
# ref = refs[0][0]
# plot_example(cand, ref, lang="en", rescale_with_baseline=False)